姓名：小宝  
学号：20202020

<h1 align='center'> 自然语言处理 </h1>

<h6> 目录 </h6>

- [处理文本](#text)
    - [练习1. 词袋](#bag)
    - [练习2. TF-IDF](#tfidf)
- [过拟合](#overfit)
    - [练习3. 查找过拟合](#lookforoverfit)


<h2> 处理文本 </h2>

到目前为止，我们一直在使用数字数据：在预测小木屋（芬兰的"mökki"）的价格时，我们已经看了小木屋的大小，室内厕所的数量等等。所有这些品质都可以用数字来表达。但是，如果我们也能获得小木屋的文字描述呢？例如，可以有前房客的评论。 "美丽的湖边位置，周围有很多自然风光！ "，或者 "小木屋就在飞行路线下，非常嘈杂！ "这可能是有价值的信息，但我们如何使用它？

赋予文本一个计算机可以使用的表示法是自然语言处理（ NLP）的基础。 NLP包括使我们能够解决各种任务的技术，如互联网搜索、文件分类和自动回答问题（想想Siri和Alexa这样的智能助手）。总的来说，计算机是计算数字的机器。因此，为了处理文本数据，我们需要找到一种将其编码为数字的方法。一种方法是使用 "词袋 "方法，即我们只需计算每个词在文件中出现的次数。

下面我们展示了一个简单的词袋矩阵的例子，每一行都描述了一首童谣中的一句话，这是来自于英语童谣的《这只小猪》。（我们之所以使用英文而不是中文是因为英文用空格间隔单词，所以我们很容易分辨"this pig"是2个单词，但是中文“这只猪”，我们需要首先判断“这只”是连在一起的。所以在自然语言处理中，中文的处理比英文更难。

In [ ]:
# 运行本段代码可以欣赏“这只小猪”童谣。

import io
import base64
from IPython.display import HTML

video = io.open('this little pig.mp4', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" controls width=720 height=480>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))


**歌词**

<pre>This little piggy went to market,
This little piggy stayed home,
This little piggy had roast beef,
This little piggy had none,
This little piggy cried "Wee! Wee! Wee!" all the way home.</pre>

<table class="css-5l5ilv ejs27q00"><tbody><tr><th>this</th><th>home</th><th>way</th><th>to</th><th>cried</th><th>wee</th><th>went</th><th>all</th><th>piggy</th><th>stayed</th><th>had</th><th>none</th><th>beef</th><th>and</th><th>the</th><th>roast</th><th>market</th><th>little</th></tr><tr><td>1</td><td>0</td><td>0</td><td>1</td><td>0</td><td>0</td><td>1</td><td>0</td><td>1</td><td>0</td><td>0</td><td>0</td><td>0</td><td>0</td><td>0</td><td>0</td><td>1</td><td>1</td></tr><tr><td>1</td><td>1</td><td>0</td><td>0</td><td>0</td><td>0</td><td>0</td><td>0</td><td>1</td><td>1</td><td>0</td><td>0</td><td>0</td><td>0</td><td>0</td><td>0</td><td>0</td><td>1</td></tr><tr><td>1</td><td>0</td><td>0</td><td>0</td><td>0</td><td>0</td><td>0</td><td>0</td><td>1</td><td>0</td><td>1</td><td>0</td><td>1</td><td>0</td><td>0</td><td>1</td><td>0</td><td>1</td></tr><tr><td>1</td><td>0</td><td>0</td><td>0</td><td>0</td><td>0</td><td>0</td><td>0</td><td>1</td><td>0</td><td>1</td><td>1</td><td>0</td><td>0</td><td>0</td><td>0</td><td>0</td><td>1</td></tr><tr><td>1</td><td>1</td><td>1</td><td>0</td><td>1</td><td>3</td><td>0</td><td>1</td><td>1</td><td>0</td><td>0</td><td>0</td><td>0</td><td>1</td><td>1</td><td>0</td><td>0</td><td>1</td></tr></tbody></table>

"词袋 "这个名字来自于这样一个事实，即这些词的书写顺序没有任何用处，只有这些词和它们出现的次数很重要。 这就好像句子中的所有单词都被混杂在一个大袋子里，以至于它们被装进袋子的顺序完全消失。忽略单词顺序的动机是，可能的单词数量已经很庞大了，要学习涉及数千个参数的模型已经很有挑战性。如果 "这只小猪 "的出现是指 "小猪这只 "以外的东西，那么模型往往需要更多的参数。所以我们将从一个简单的词袋方法开始。 

你可能还记得课堂中我们讨论过的垃圾邮件过滤器：它是基于朴素贝叶斯方法，涉及到与信息中的单词相对应的可能性比率相乘。你可以把它看作是一个词袋技术，因为如果你以朴素贝叶斯的方式进行计算，无论给定哪个词的顺序，其结果都是一样的。

使用词袋表示法的一个简单例子是相似性比较，例如，我们可以在最近邻方法中使用，以确定任何两行文本之间的曼哈顿距离。

<h4> 练习1 词袋 <a id='bag'> </a></h4>
<h5> 练习1.1 </h5>

考虑上表，其中包含 This Little Piggy 童谣的词袋表示。 每一行都是一句童谣的词袋计算。

请逐字比较各行，并将每个字的计数差异相加。 例如，前两行之间的差异（距离）变为

$$|1–1| + |0–1| + |0–0| + |1–0| + ... + |1–1| = 0 + 1 + 0 + 1 + ... + 0 = 5$$

$|·| $标记绝对值，例如 |0–1| = 1。

哪两条线彼此最相似？

- 第一个和第二个
- 第一个和第三个
- 第一个和第四个
- 第一个和第五个
- 第二个和第三个
- 第二个和第四个
- 第三个和第五个
- 第三个和第四个
- 第四个和第五个

In [ ]:
# 你的答案是

<h5> 练习1.2 </h5>

你的任务是编写一个函数来计算 This Little Piggy 童谣中一对句子之间的距离（或差异）。

列表数据中的每一行代表句子（歌词）中的一行。

运行代码时，你会看到整个程序的输出是一个列表。 如果函数正常工作，每个列表将包含数据中单行与所有其他行之间的距离。

请注意，程序会将每一行也与自身进行比较。 在这种情况下——当比较的行相同时——它们的距离将为零。

你可以使用函数 `abs(x-y)` 计算数字 `x` 和 `y` 之间的距离，其中` x` 来自列表第 $1$ 行，`y` 来自第 $2$ 行。

你的程序必须适用于任何文本，而不仅仅是针对 This Little Piggy。

In [ ]:
#这个数据是《这只小猪》的词袋表示法
data = [[1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1],
        [1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1],
        [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1],
        [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1],
        [1, 1, 1, 0, 1, 3, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1]]
def distance(row1, row2):
    # 修复这个函数，使其返回在行1和行2中出现的每个词的差异之和
    # 你可以假设row1和row2是长度相等的列表，包含数字值
    return 0
def all_pairs(data):
    # 这将调用数据中所有两行组合的距离函数
    # 你不需要改变这个
    dist = [[distance(sent1, sent2) for sent1 in data] for sent2 in data]
    print(dist)
    
all_pairs(data)


In [ ]:
import numpy as np
a = [1,2]
b = [3,4]
sum(np.subtract(a,b))

<h5> 练习1.3 <a id='1.3'></a></h5> 

你的任务是编写一个程序，计算《这只小猪》儿歌中每一对句子之间的距离（或差异），并找出最相似的一对 。使用[曼哈顿距离](#manhattan)（也叫出租车距离）作为你的距离指标。 

你可以先建立一个`numpy`数组来存储所有的距离。注意，数组中的对角线元素（位于`[i, j]`位置的元素， `i=j`）将等于零。 发生这种情况是因为程序会将每一行也与自己进行比较。为了避免选择这些元 素，你可以赋值为`np.inf`（可能的最大浮点值）。要做到这一点，必须确保数组的类型是浮点数。 获取二维数组（或事实上任何维度）中数值最低的元素的索引的一个快速方法是通过函数 `np.unravel_index(np.argmin(dist), dist.shape)) `其中`dist`是二维数组。这将把索引作为一个长度为2的列表返回。

参考[这里](#unravel)查看更多关于`np.unravel_index`的解释。

In [ ]:
import numpy as np

data = [[1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1],
        [1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1],
        [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1],
        [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1],
        [1, 1, 1, 0, 1, 3, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1]]

def find_nearest_pair(data):
    N = len(data)
    dist = np.empty((N, N), dtype=np.float)
    # 此处补充代码
    
    print(np.unravel_index(np.argmin(dist), dist.shape))

find_nearest_pair(data)

# 输出结果应该是(2, 3)

词袋方法的一个问题是我们基本上对任何单词都给予同等重视：单词“a”出现十次与单词“great”出现十次一样重要。 但是，例如，如果我们考虑文本情感（分析某人的感受），“a”和“great”显然并不同等重要。

我们可以应用一种流行且有效的解决方案来改进简单的词袋表示，其首字母缩写为 tf-idf。

<h3> TF-IDF </h3>

TF-IDF是一种用于文本数据中词语加权的常用技术。TF代表词频（Term Frequency），指某个词在文本中出现的次数；IDF代表逆文档频率（Inverse Document Frequency），指与该词相关的文档数的倒数的对数。将这两个因素相乘，可以得到一个词语在文本中的重要性权值。

在使用TF-IDF技术对文本进行处理时，词语的权值会随着它在文本中出现的频率增加而增加，但同时也会随着它在整个语料库中出现的频率增加而减少。因此，TF-IDF可以帮助我们找到在某个文本中最常见但同时又是在整个语料库中不常见的词语，从而更好地理解文本的主题和内容。在文本分类、搜索引擎和信息检索等任务中，TF-IDF都是常用的文本特征提取技术。

在其最简单的形式下，该算法如下。

1. 计算你的文档集合中每个词的频率（出现的次数除以文档长度）。这就是 "术语频率"，或称**tf**。
2. 计算每个词出现在多少个文件中，然后将其除以文件的总数。这就是"文档频率"， 简称**df**。由于我们希望给常见的词分配较少的权重，我们将使用这个的倒数， $1÷df$。
3. 有不同的方法来结合这两个数字来给每个词分配权重。最常见的是术语频率与文档频率倒数的乘积： 
$$tf-idf = tf ×log(1 ÷ df )$$

这可能听起来很复杂，但不要惊慌！通过下面的例子会更有意义。通过下面的例子，它
将变得更有意义。让我们考虑一下这三个句子。
1. "He really, really loves coffee"
2. "My sister dislikes coffee"
3. "My sister loves tea"

在tf-idf术语中，每个句子都是一个文档，它们共同构成了我们的语料库。为了计算每个文档的tfidf表示法，我们首先列出我们在语料库中的所有独特词汇。在这种情况下，它们是 'coffee', 'dislikes', 'he', 'loves', 'my', 'really', 'sister', 和 'tea'。（忽略大小写）

接下来我们依次查看每个文件，计算其中每个词的出现次数，然后用这个数字除以文件的长度。记住：频率是指出现的比率，所以出现的次数除以文档中的总字数。

1. he:$\frac{1}{5}$，really:$\frac{2}{5}$，loves:$\frac{1}{5}$，coffee:$\frac{1}{5}$
2. my:$\frac{1}{4}$，sister:$\frac{1}{4}$，dislikes:$\frac{1}{4}$，coffee:$\frac{1}{4}$
3. my:$\frac{1}{4}$，sister:$\frac{1}{4}$，loves:$\frac{1}{4}$，tea:$\frac{1}{4}$


已经成功了一半! 接下来， 让我们来计算每个词的文档频率。一个词的文档频率是指包含该词的至少一次出现的文档的数量。
coffee: $\frac{2}{3}$，dislikes:$\frac{1}{3}$，he:$\frac{1}{3}$，loves:$\frac{2}{3}$，my:$\frac{2}{3}$，really:$\frac{1}{3}$，sister:$\frac{2}{3}$，tea:$\frac{1}{3}$

现在剩下的就是计算每个文档中每个词的tf-idf得分，方法是将术语频率与文档频率的对数相乘。

你应该尝试自己计算这些数字。为了得到对数，你需要一个带有 "对数"键的老式计算器或电子表格应用程序（只需输入例如`=log(3)`就可以得到$0.4771212547$的数值）。你甚至可以在许多网络浏览器中进行数学运算，只需在搜索栏中输入公式。

对数的基数，不是很重要，但为了得到与我们的例子和练习中相同的数字， 你应该使用以10为基数的对数。这似乎是许多电子表格应用程序和浏览器的默认设置，但在桌面计算器或编程语言（如Python）中却不是。如果你使用Python，你可以通过使用函数`math.log(x, 10)`得到`x`基数为10的对数。

以第一句中的 "he "这个词为例。由于该词出现过一次，所以词频为$\frac{1}{5}$， 而文件频率为$\frac{1}{3}$。所以其$tf-idf=\frac{1}{5} \times log(\frac{3}{1}) \approx 0.095$

下面是第一句"he really really loves coffee"中所有单词的tf-idf计算结果。

| | | | |
| -| - | -| -|
|he| $tf=\frac{1}{5}$ | $log(\frac{1}{df})=log(\frac{3}{1})$ | $tf-idf=\frac{1}{5} \times log(\frac{3}{1}) = 0.095$ |
|really|$tf=\frac{2}{5}$ | $log(\frac{1}{df})=log(\frac{3}{1})$ | $tf-idf=\frac{2}{5} \times log(\frac{3}{1}) = 0.1908$ |
|loves|$tf=\frac{1}{5}$ | $log(\frac{1}{df})=log(\frac{3}{2})$ | $tf-idf=\frac{1}{5} \times log(\frac{3}{2}) = 0.0352$ |
|coffee|$tf=\frac{1}{5}$ | $log(\frac{1}{df})=log(\frac{3}{2})$ | $tf-idf=\frac{1}{5} \times log(\frac{3}{2}) = 0.0352$ |


In [ ]:
css = """
div.special_table + table * {
  border: 1px solid black;
}
"""

pn.extension(raw_css=[css])

pn.panel("""
<div class="special_table"></div>
| | | | |
| -| - | -| -|
|he| $$tf=\frac{1}{5}$$ | $$log(\frac{1}{df})=log(\frac{3}{1})$$ | $$tf-idf=\frac{1}{5} \times log(\frac{3}{1}) = 0.095$$ |
|he| $$tf=\frac{1}{5}$$ | $$log(\frac{1}{df})=log(\frac{3}{1})$$ | $$tf-idf=\frac{1}{5} \times log(\frac{3}{1}) = 0.095$$ |
|he| $$tf=\frac{1}{5}$$ | $$log(\frac{1}{df})=log(\frac{3}{1})$$ | $$tf-idf=\frac{1}{5} \times log(\frac{3}{1}) = 0.095$$ |
|he| $$tf=\frac{1}{5}$$ | $$log(\frac{1}{df})=log(\frac{3}{1})$$ | $$tf-idf=\frac{1}{5} \times log(\frac{3}{1}) = 0.095$$ |
""")

tf-idf背后的逻辑是合理的：如果一个词在所有的文档中都很常见（例如 "the"、 "is"、 "a"、 "and"等），那么在描述一个文档时，即使它在文档中出现很多次，也不会有很大的信息量。另一方面，如果一个词在语料库中非常罕见（ "表观遗传学"， "梯度"），即使在一个文件中出现一次，也可能是有信息的。

<h4> 练习2 tf-idf <a id='tfidf'></a> </h4>
<h5> 练习2.1 </h5>

我们还是使用一首英文童谣Humpty Dumpty来练习tf-idf的计算。

![humpty](richdiesslin_humpty_untold_ani.gif)

<pre>
Humpty Dumpty sat on a wall,
Humpty Dumpty had a great fall.
All the king's horses and all the king's men
Couldn't put Humpty together again.
</pre>

Humpty Dumpty 第 1 行中单词“Humpty”的词频（tf）是多少？
- 1
- 1/6
- 1/4

Humpty Dumpty 第 3 行中单词“all”的词频（tf）是多少？
- 1
- 2/9
- 1/9

Humpty Dumpty中“Humpty”的文档频率（df）是多少？
- 3/4
- 3
- 4/3

Humpty Dumpty 第 4 行中单词“Humpty”的tf-idf是多少？
- ~0.02
- ~0.22
- 3/4

<h5> 练习2.2 </h5>

修改以下程序以打印出每个文档和每个单词的 tf-idf 值。 以下代码计算 tf 和 df 值，因此你只需根据正确的公式将它们组合起来即可。 共有三个文档（句子）和总共八个术语（唯一词），因此输出应该是三个列表，每个列表包含八个 tf-idf 值。

In [ ]:
# 文本数据

text = '''he really really loves coffee
my sister dislikes coffee
my sister loves tea'''

import math

def main(text):
    # 首先将文本分成行，然后分别将每行拆分成单词列表
    docs = [line.split() for line in text.splitlines()]

    N = len(docs)

    # 创建词汇表：至少出现一次的单词列表
    vocabulary = list(set(text.split()))

    df = {}
    tf = {}
    for word in vocabulary:
        # tf: 单词w在文档中出现次数除以文档长度
        # 注意：tf[word] 将是一个包含每个文档中每个单词的tf的列表
        # 例如 tf['he'][0] 包含单词 'he' 在第一个文档中的词频
        tf[word] = [doc.count(word)/len(doc) for doc in docs]

        # df: 包含单词w的文档数量
        df[word] = sum([word in doc for doc in docs])/N

    # 遍历文档计算tf-idf值
    for doc_index, doc in enumerate(docs):
        tfidf = []
        for word in vocabulary:
            # 添加正确的公式在这里。请记住使用以10为底的对数：math.log(x, 10)
            tfidf.append(None) 

        print(tfidf)

main(text)


<h5> 练习2.3 </h5>

让我们把两个任务结合起来：寻找最相似的一对线和tf-idf表示法。

编写一个程序，使用tfidf向量来寻找给定数据集中最相似的一对句子。你可以用下面的例子文本测试你的解决方案。但是请注意，你的解决方案也将在其他数据集上进行测试，所以请确保你不利用例子数据的任何特殊属性（比如有四行文字）。

这个练习需要比一般的工作多一点，但你应该能从前面的练习中受益。

In [ ]:
text = '''Humpty Dumpty sat on a wall
Humpty Dumpty had a great fall
all the king's horses and all the king's men
couldn't put Humpty together again'''

def main(text):
    # 你的代码应该执行的任务。
    # 1. 将文本分割成单词，并获得其中出现的独特单词的列表
    #      一个简短的单行代码，将文本分割成句子（令单词全部小写）
    #      docs = [line.lower().split() for line in text.split('\n')] 
    # 2. 仔细检查每一个词，并计算它的词频和它的文档频率（多次出现的同一单词只计算一次）
    # 3.在你有了术语频率和文档频率之后，去看文本中的每一行，然后计算其TF-IDF表示，这将是一个向量
    # 4. 在计算了文本中每一行的TF-IDF表示之后，你需要计算每一行之间的距离，找出最接近的。

main(text)


在你获得了文件的向量表示后，你可以以许多不同的方式利用它们。一些经典的文本分类的例子包括检测产品评论的情绪（正面/负面）和电子邮件的垃圾邮件（垃圾邮件/非垃圾邮件）。

但这一切有什么用呢？这里有一个例子可以说明：对于特定语料库中的任何给定句子，你现在有一个向量表示。你是否注意到与我们在上一次练习中表示小木屋的方式有相似之处？对于每个小屋，我们有一个向量，其组成部分以某种方式描述它。 现在对于一个句子，我们有一个向量，其组成部分以某种方式描述了该句子。重点是我们现在可以对文本进行分类，因为它是由数字而不是文字来表示的。

另一种获得文档矢量表示的方法是依靠单词嵌入，它是单词的密集实数矢量表示。我们不会在这里详细介绍这些是如何产生的，但有兴趣的同学可以搜索 "word2vec"

<h2> 过拟合 <a id='overfit'></a><h2>

在我们去构建任何很酷的机器学习应用之前， 我们需要认真讨论一个叫做过拟合的东西。 过拟合是一个基本问题， 应该是任何实用的机 器学习课程的必修部分。不了解它，就会造成很大的伤害。那么，它是什么呢？ 

简单地说，过拟合意味着对在训练数据中有效的预测过于自信。 我们在上面讨论线性回归和小屋价格时顺便提到了训练数据和测试数据的区别。一个有5个预测变量（如小木屋的大小、桑拿房的大小、与湖泊的距离等）的线性模型，在包括三个小木屋的训练数据中调整预测价格， 会尽职尽责地完全复制这三个价格。当有10个预测变量和10个小木屋时，以及在任何预测变量的数量与样本大小相同的情况下，也是如此。不过，很明显，训练数据中的完美 "预测 "并不能保证对任何其他数据的完美预测。从本质上讲，这是一个过度拟合的极端案例。 

另一个同样极端的过拟合案例发生在最近邻方法上。再回顾一下小屋定价问题。我们有一些关于小屋销售的训练数据，我们知道小屋的细节和支付价格。让我们看看最近邻方法是如何预测其中任何一个的价格的。假设其中一个木屋是我们上面例子数据中的第一个。 66平方米，一个5平方米的桑拿房，离湖边15米，有2个室内卫生间，离邻居500米，标价258,250元。请注意，这个小屋是我们的 "测试数据 "，但它也包括在训练数据中。如果我们把小屋的细节反馈给最近邻方法，它将简单地在训练数据中找到完全相同的小屋，并确定它是测试数据点的最近的邻居。因此，它将预测价格为258,250元。由于这是测试数据中的价格，我们发现这个价格被准确预测了。 训练数据中的任何其他小屋也是如此 。 

正如你所注意到的，当我们在训练数据中加入比预测变量更多的船舱时（在我们的例子中是5个），线性模型就不能再完美地拟合训练数据。换句话说，我们会说训练误差不为零： 当预测变量的数量少于任何线性模型的样本量时， 这种情况几乎总是发生（除非数据恰好非常特殊）。我们将在深度学习中研究非线性模型，允许更多的灵活性， 但即使样本量大于预测变量的数量， 我们仍然可能得到一个非常小的训练误差。


这里最重要的观察是，一个小的训练误差，特别是如果它是通过对一个小的训练数据集拟合一个复杂的、可能是非线性的模型得到的， 并不能保证该模型实际上能很好地预测新的数据。 事实上，像我们的线性模型或近邻方法那样的零训练误差，在与训练数据不重合的测试数据上，可能仍然会出现非常差的预测精度。所以我们已经确定，过度拟合不是一件好事。

那么我们如何避免它呢？第一道防线是一些人可能已经在练习中熟悉的东西：将你的数据分成训练和测试数据。当你用原始数据的一部分训练模型，并用另一部分测试其性能时，你至少会对你的模型在使用未见过的数据时的泛化程度有一些了 解。你可能仍然有挥之不去的疑虑：分割的方式好吗？如果以另一种方式分割数据，模型的性能是否会有很大的不同？ 

一个简单的解决方案是将数据分成n个不同的集合， 然后训练模型n次--每次用n-1个集合的不同组合， 剩下的集合作为测试集合。 这样， 你 将得到n个关于你所选模型在使用未见过的数据时的表现的估计。这被称为 "留一 "交叉验证，它是交叉验证的最简单的方法之一。 由于过拟合是机器学习领域的一个痛点，很多人都花时间和精力来想办法对付它。其中大部分都不在本课程的范围内，但有兴趣的同学可以查一下*正则化*和*dropout*等方法，这两种方法都是在线性和逻辑回归以及神经网络中被广泛理解和使用的例子。


<h4> 练习3 查找过拟合 <a id='lookforoverfit'></a></h4>
<h5> 练习3.1 </h5>

让我们暂时假设你有一个数据集，包含 1000 封标记为垃圾邮件或非垃圾邮件的电子邮件。 在 1000 封邮件中，990 封是合法电子邮件，10 封是垃圾邮件。

然后，你将数据分成训练集和测试集，使两个标签以相等的比例出现在两个集合中，然后在数据上训练分类器。

你会将什么设置为你的模型必须超越才能被认为有价值的基准准确度？
- 50%，随机抽取垃圾或非垃圾邮件
- 99%，选择大多数类别
- 33.3%

<h5> 练习3.2 </h5>

下面的程序使用了所谓的kNN算法。这个想法是不仅要看单个最近的训练数据点（邻居），而且要看例如五个最近的点，如果k=5。正常的近邻分类器相当于使用k=1。

该程序根据k值进行分类，并输出一个图像，显示不同事物的分类情况。修改程序，使程序打印出训练和测试的准确性，并使之能够使用不同的k值。

提示：你可以用函数`knn.score`得到模型的准确度。例如：`knn.score(x_train, y_train)`在你首先通过调用`knn.fit(x_train, y_train)`创建分类器后，返回训练集的精度。尝试不同的k值来回答下面的问题。

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
import numpy as np

# 创建模拟数据集
x, y = make_moons(
    n_samples=500,  # 样本数
    random_state=42,
    noise=0.3
)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

# 创建一个分类器，并将其适用于我们的数据
knn = KNeighborsClassifier(n_neighbors=50)
knn.fit(x_train, y_train)
train_acc = knn.score(x_train, y_train)
print("训练准确率: %f" % train_acc)
test_acc = knn.score(x_test, y_test)
print("测试准确率: %f" % test_acc)

In [ ]:
# 运行代码查看图像，修改上方knn = KNeighborsClassifier(n_neighbors=50)中50
# 重新运行本代码查看区别
%run -i Lab5_vknn.py

你认为下面哪个k值“最好”？
- k = 1
- k = 250
- k = 42
- k = 100
- 无所谓

为什么？
- 给出了最低的训练准确度
- 给出了最高的训练准确度
- 给出了最高的测试准确度
- 给出了最低的测试准确度
- k的选择并不重要

In [ ]:
# 你的答案是

<h5> 练习3.3 </h5>

为了使你的模型被认为是有用的，你的模型应该超过什么合理的基线精度（或者说，准确率的最低标准应该是多少？
- 0.50
- 0.25
- 任何优于所有错误的表现都足以作为基线 

是否有可能出现测试集准确率高于训练集准确率的情况？ 
- 是
- 否

In [ ]:
# 你的答案是

In [ ]:
# 恭喜你完成实验五的练习

<details>
    <summary><h6>点击查看答案</h6></summary>
    这是可能的，并且有很多原因。 例如，如果你正在做一个像这里这样的分类任务，如果你的数据集有类不平衡，它很容易导致这种情况，或者如果你在这个例子中的测试集点在远离决策边界的地方被选中，那么它们 比边界附近的那些更容易正确分类。</details>

**曼哈顿距离** <a id='manhattan'></a>
![dist](taxicab-manhattan-grid-layout-paths.gif)

计程车几何（Taxicab geometry）或曼哈顿距离（英语：Manhattan distance/Manhattan length）或方格线距离是由十九世纪的赫尔曼·闵可夫斯基所创辞汇，为欧几里得几何度量空间的几何学之用语，用以标明两个点上在标准坐标系上的绝对轴距之总和。

我们可以定义曼哈顿距离的正式意义为L1-距离或城市区块距离，也就是在欧几里得空间的固定直角坐标系上两点所形成的线段对轴产生的投影的距离总和。

例如在平面上，坐标（x1, y1）的点P1与坐标（x2, y2）的点P2的曼哈顿距离为：

$$d(x,y)=\left|x_{1}-x_{2}\right|+\left|y_{1}-y_{2}\right|$$

要注意的是，曼哈顿距离依赖座标系统的旋转，而非系统在座标轴上的平移或映射。

曼哈顿距离的命名原因是从规划为方型建筑区块的城市（如美国纽约市曼哈顿区）间，最短的行车路径而来（忽略曼哈顿的单向车道以及只存在于3、14大道的斜向车道）。任何往东三区块、往北六区块的的路径一定最少要走九区块，没有其他捷径。

[返回](#1.3)

**np.unravel_index** <a id='unravel'></a>

计算机内存是线性寻址的。 每个存储单元对应一个数字。 一个内存块可以根据基址（它的第一个元素的内存地址）和项目索引来寻址。 例如，假设基地址为 10000：

|项目索引 |0| 1| 2| 3|
|-|-|-|-|-|
|内存地址| 10000| 10001| 10002 |10003|

要存储*多维*块，必须以某种方式使它们的几何形状*适合线性*内存。 在 `C` 和 `NumPy` 中，这是逐行完成的。 一个二维示例是：

| |0| 1| 2| 3|
|-|-|-|-|-|
|**0** | 0| 1| 2| 3|
|**1** | 4| 5| 6| 7|
|**2** | 8 |9 |10| 11|

因此，例如，在这个 3×4 大小的块中，二维索引 (1, 2) 将对应于线性索引$ 6$，即 $1 \times 4 + 2$。

`unravel_index` 做相反的事情。 给定一个线性索引，它计算相应的*多为*索引。 因为这取决于块尺寸，所以需要传递该信息给函数。 因此，在我们的示例中，我们可以从线性索引 6 返回原始二维索引 (1, 2)：

```python
>>> np.unravel_index(6, (3, 4)) # 其中6是我们已知的1维索引，（3，4）是我们的块大小
(1, 2)                 # (1,2)就是6在二维数组中的位置。记住我们总是从0开始数
```

注意：以上内容掩盖了一些细节。 1) 将项目索引转换为内存地址还必须考虑项目大小。 例如，一个整数通常有 4 或 8 个字节。 因此，在后一种情况下，项目 `i` 的内存地址将为 `base + 8 x i`。 2). `NumPy` 比建议的要灵活一些。 如果需要，它可以逐列组织多维数据。 它甚至可以处理内存中不连续的数据，例如留有间隙等。

[返回](#1.3)